# Relationship Classification — Analysis

This notebook has **two parts per label**:

1. **Pre-training** — dataset statistics, class balance, text characteristics, co-occurrence.
2. **Post-training** — grid search results, test metrics, confusion matrices, error analysis.

Run the cells top-to-bottom. The post-training sections load from `results/` and will be skipped if the pipeline hasn't been run yet.

---
## Setup

In [ ]:
import os
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import display
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

from training.config import (
    DATASET_PATH,
    DATASET_SHEET,
    FINAL_EVAL_CSV,
    GRID_SEARCH_CSV,
    LABEL_TRUE_VALUES,
    LABELS,
    PREDICTIONS_SUBDIR,
    RESULTS_DIR,
    TEXT_COLUMN,
)

warnings.filterwarnings("ignore", category=FutureWarning)
sns.set_theme(style="whitegrid", palette="muted")
pd.set_option("display.max_colwidth", 120)

%matplotlib inline

---
## Load Dataset

In [ ]:
ext = os.path.splitext(DATASET_PATH)[1].lower()
if ext == ".xlsx":
    df = pd.read_excel(DATASET_PATH, sheet_name=DATASET_SHEET)
else:
    df = pd.read_csv(DATASET_PATH)

for label in LABELS:
    df[label] = df[label].apply(lambda v: v in LABEL_TRUE_VALUES)

df = df.dropna(subset=[TEXT_COLUMN]).reset_index(drop=True)
df["text_len"] = df[TEXT_COLUMN].str.len()
df["word_count"] = df[TEXT_COLUMN].str.split().str.len()
df["n_labels"] = df[LABELS].sum(axis=1).astype(int)

print(f"Dataset: {len(df):,} rows, {len(df.columns)} columns")
df.head(3)

---
## Load Results (if available)

In [ ]:
grid_path = os.path.join(RESULTS_DIR, GRID_SEARCH_CSV)
eval_path = os.path.join(RESULTS_DIR, FINAL_EVAL_CSV)

HAS_RESULTS = os.path.isfile(eval_path)

if HAS_RESULTS:
    grid_df = pd.read_csv(grid_path) if os.path.isfile(grid_path) else None
    eval_df = pd.read_csv(eval_path)
    preds = {}
    for label in LABELS:
        p = os.path.join(RESULTS_DIR, PREDICTIONS_SUBDIR, f"{label}_predictions.csv")
        if os.path.isfile(p):
            preds[label] = pd.read_csv(p)
    print(f"Results loaded: {len(eval_df)} labels evaluated.")
    if grid_df is not None:
        print(f"Grid search: {len(grid_df)} trials.")
else:
    grid_df = None
    eval_df = None
    preds = {}
    print("No results found — post-training sections will be skipped.")
    print("Run `python train.py` first, then re-run this notebook.")

---
## Global Overview

### Label Distribution

In [ ]:
counts = df[LABELS].sum().sort_values(ascending=False)
pcts = (counts / len(df) * 100).round(1)

summary = pd.DataFrame({"count": counts, "pct": pcts})
print(summary.to_string())

fig, ax = plt.subplots(figsize=(8, 4))
bars = ax.barh(counts.index[::-1], counts.values[::-1])
for bar, pct in zip(bars, pcts.values[::-1]):
    ax.text(
        bar.get_width() + len(df) * 0.01,
        bar.get_y() + bar.get_height() / 2,
        f"{pct}%",
        va="center",
        fontsize=10,
    )
ax.set_xlabel("Count")
ax.set_title("Label Distribution")
plt.tight_layout()
plt.show()

### Labels per Example

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3))
df["n_labels"].value_counts().sort_index().plot.bar(ax=ax)
ax.set_xlabel("Number of labels per example")
ax.set_ylabel("Count")
ax.set_title("Multi-label Distribution")
plt.tight_layout()
plt.show()

### Co-occurrence Matrix

In [ ]:
co = pd.DataFrame(index=LABELS, columns=LABELS, dtype=int)
for l1 in LABELS:
    for l2 in LABELS:
        co.loc[l1, l2] = int((df[l1] & df[l2]).sum())

fig, ax = plt.subplots(figsize=(7, 6))
sns.heatmap(co.astype(int), annot=True, fmt="d", cmap="Blues", ax=ax, square=True)
ax.set_title("Label Co-occurrence")
plt.tight_layout()
plt.show()

### Text Length Overview

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

df["text_len"].hist(bins=50, ax=axes[0], edgecolor="white")
axes[0].set_xlabel("Character length")
axes[0].set_ylabel("Count")
axes[0].set_title("Text Length (characters)")
axes[0].axvline(
    df["text_len"].median(),
    color="red",
    linestyle="--",
    label=f"median={df['text_len'].median():.0f}",
)
axes[0].legend()

df["word_count"].hist(bins=50, ax=axes[1], edgecolor="white")
axes[1].set_xlabel("Word count")
axes[1].set_ylabel("Count")
axes[1].set_title("Text Length (words)")
axes[1].axvline(
    df["word_count"].median(),
    color="red",
    linestyle="--",
    label=f"median={df['word_count'].median():.0f}",
)
axes[1].legend()

plt.tight_layout()
plt.show()

### Global Test Results Summary

In [ ]:
if HAS_RESULTS and eval_df is not None:
    display_cols = [
        c
        for c in [
            "label",
            "test_f1",
            "test_precision",
            "test_recall",
            "test_accuracy",
            "n_test",
            "n_test_positive",
            "gen_model",
            "reflection_model",
        ]
        if c in eval_df.columns
    ]
    display(eval_df[display_cols])

    fig, ax = plt.subplots(figsize=(8, 4))
    metrics = eval_df.set_index("label")[["test_f1", "test_precision", "test_recall"]]
    metrics.plot.bar(ax=ax)
    ax.set_ylabel("Score")
    ax.set_title("Test Metrics by Label")
    ax.set_ylim(0, 1.05)
    ax.legend(loc="lower right")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()
else:
    print("Skipped — no results.")

---
---
## Per-Label Analysis

Each label section below contains:
- **Pre-training**: class balance, text length by class, sample examples.
- **Post-training**: grid search heatmap, test confusion matrix, error examples.

In [ ]:
def pre_training_analysis(label):
    """Pre-training statistics for a single label."""
    pos = df[df[label]]
    neg = df[~df[label]]
    n_pos, n_neg = len(pos), len(neg)
    print(
        f"Positive: {n_pos} ({n_pos / len(df) * 100:.1f}%)  |  "
        f"Negative: {n_neg} ({n_neg / len(df) * 100:.1f}%)"
    )
    print(f"Imbalance ratio: 1:{n_neg / n_pos:.1f}" if n_pos > 0 else "No positives!")
    print()

    # Text length comparison
    fig, axes = plt.subplots(1, 2, figsize=(12, 3.5))
    for ax, col, title in [
        (axes[0], "text_len", "Char length"),
        (axes[1], "word_count", "Word count"),
    ]:
        bins = np.linspace(df[col].min(), df[col].quantile(0.95), 40)
        ax.hist(neg[col], bins=bins, alpha=0.6, label="Negative", edgecolor="white")
        ax.hist(pos[col], bins=bins, alpha=0.6, label="Positive", edgecolor="white")
        ax.set_xlabel(title)
        ax.set_ylabel("Count")
        ax.legend()
        ax.set_title(f"{title} — positive vs negative")
    plt.suptitle(f"'{label}' — text length distributions", fontsize=12, y=1.02)
    plt.tight_layout()
    plt.show()

    # Co-occurring labels for positives
    if n_pos > 0:
        other_labels = [lb for lb in LABELS if lb != label]
        co_counts = pos[other_labels].sum().sort_values(ascending=False)
        co_pcts = (co_counts / n_pos * 100).round(1)
        print("Co-occurring labels among positives:")
        for name, count in co_counts.items():
            if count > 0:
                print(f"  {name}: {count} ({co_pcts[name]}%)")
        print()

    # Sample examples
    n_show = min(3, n_pos)
    if n_show > 0:
        print(f"Sample positive examples (n={n_show}):")
        for _, row in pos.sample(n=n_show, random_state=42).iterrows():
            print(f"  - {row[TEXT_COLUMN][:200]}")
    print()


def post_training_analysis(label):
    """Post-training results for a single label."""
    if not HAS_RESULTS or eval_df is None:
        print("Skipped — no results.")
        return

    # Grid search heatmap
    if grid_df is not None and label in grid_df["label"].values:
        label_grid = grid_df[grid_df["label"] == label]
        pivot = label_grid.pivot_table(
            index="gen_model", columns="reflection_model", values="val_f1"
        )
        fig, ax = plt.subplots(figsize=(7, 4))
        sns.heatmap(
            pivot,
            annot=True,
            fmt=".3f",
            cmap="YlOrRd",
            ax=ax,
            vmin=0,
            vmax=1,
            square=True,
        )
        ax.set_title(f"'{label}' — Grid Search Val F1")
        plt.tight_layout()
        plt.show()

    # Test metrics
    row = eval_df[eval_df["label"] == label]
    if row.empty:
        print(f"No evaluation row for '{label}'.")
        return
    row = row.iloc[0]
    print(f"Test F1:        {row['test_f1']:.4f}")
    print(f"Test Precision: {row['test_precision']:.4f}")
    print(f"Test Recall:    {row['test_recall']:.4f}")
    print(f"Test Accuracy:  {row['test_accuracy']:.4f}")
    print(
        f"Test size:      {int(row['n_test'])} ({int(row['n_test_positive'])} positive)"
    )
    if "gen_model" in row.index:
        print(f"Best gen model:        {row['gen_model']}")
        print(f"Best reflection model: {row['reflection_model']}")
    print()

    # Confusion matrix
    if label in preds:
        pred_df = preds[label]
        y_true = (pred_df["gold"] == "true").astype(int)
        y_pred = (pred_df["predicted"] == "true").astype(int)
        cm = confusion_matrix(y_true, y_pred)

        fig, ax = plt.subplots(figsize=(4, 4))
        ConfusionMatrixDisplay(cm, display_labels=["Negative", "Positive"]).plot(
            ax=ax, cmap="Blues"
        )
        ax.set_title(f"'{label}' — Test Confusion Matrix")
        plt.tight_layout()
        plt.show()

        # Error examples
        errors = pred_df[~pred_df["correct"]]
        if len(errors) > 0:
            fn = errors[errors["gold"] == "true"]
            fp = errors[errors["predicted"] == "true"]
            print(f"Errors: {len(errors)} total — {len(fp)} FP, {len(fn)} FN")
            for err_type, err_df in [("False Positive", fp), ("False Negative", fn)]:
                n_show = min(3, len(err_df))
                if n_show > 0:
                    print(f"\n  {err_type} examples (n={n_show}):")
                    for _, r in err_df.head(n_show).iterrows():
                        print(f"    - {r['text'][:200]}")
        else:
            print("No errors on the test set!")
    print()

---
### romantic

In [ ]:
print("=" * 60)
print("PRE-TRAINING — romantic")
print("=" * 60)
pre_training_analysis("romantic")

In [ ]:
print("=" * 60)
print("POST-TRAINING — romantic")
print("=" * 60)
post_training_analysis("romantic")

---
### family

In [ ]:
print("=" * 60)
print("PRE-TRAINING — family")
print("=" * 60)
pre_training_analysis("family")

In [ ]:
print("=" * 60)
print("POST-TRAINING — family")
print("=" * 60)
post_training_analysis("family")

---
### friendship

In [ ]:
print("=" * 60)
print("PRE-TRAINING — friendship")
print("=" * 60)
pre_training_analysis("friendship")

In [ ]:
print("=" * 60)
print("POST-TRAINING — friendship")
print("=" * 60)
post_training_analysis("friendship")

---
### professional

In [ ]:
print("=" * 60)
print("PRE-TRAINING — professional")
print("=" * 60)
pre_training_analysis("professional")

In [ ]:
print("=" * 60)
print("POST-TRAINING — professional")
print("=" * 60)
post_training_analysis("professional")

---
### unknown

In [ ]:
print("=" * 60)
print("PRE-TRAINING — unknown")
print("=" * 60)
pre_training_analysis("unknown")

In [ ]:
print("=" * 60)
print("POST-TRAINING — unknown")
print("=" * 60)
post_training_analysis("unknown")

---
### irrelevant

In [ ]:
print("=" * 60)
print("PRE-TRAINING — irrelevant")
print("=" * 60)
pre_training_analysis("irrelevant")

In [ ]:
print("=" * 60)
print("POST-TRAINING — irrelevant")
print("=" * 60)
post_training_analysis("irrelevant")